In [79]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Specify the path to the Chrome Beta binary
chrome_binary_path = "/Applications/Google Chrome Beta.app/Contents/MacOS/Google Chrome Beta"

# Specify the path to the ChromeDriver executable for Chrome Beta
driver_path = "/Users/jongarcia/codeup-data-science/googlemaps-scraper/chromedriver-mac-x64 BETA/chromedriver"

# Create a ChromeOptions object
options = Options()

# Set the binary location to Chrome Beta
options.binary_location = chrome_binary_path

# Create a Service object with the ChromeDriver path
service = Service(driver_path)

# Create a ChromeDriver with the specified options and executable path
driver = webdriver.Chrome(service=service, options=options)


In [80]:
# Open Google.com
driver.get("https://www.google.com/maps/place/?q=place_id:ChIJIyiwrqX0wokR1x1J2xWMNsg")

In [53]:
driver.get("https://www.google.com/maps/place/?q=place_id:ChIJwTC7A55ZwokRPNX9g7ngbaI")

In [81]:
from bs4 import BeautifulSoup
import time

soup = BeautifulSoup(driver.page_source, 'html.parser')

place = {}

try:
    place['name'] = soup.find('h1', class_='DUwDvf lfPIob').text.strip()
except Exception:
    place['name'] = None

try:
    place['overall_rating'] = float(soup.find('div', class_='F7nice').find('span', class_='ceNzKf')['aria-label'].split(' ')[0])
except Exception:
    place['overall_rating'] = None

try:
    place['n_reviews'] = int(soup.find('div', class_='F7nice').text.split('(')[1].replace(',', '').replace(')', ''))
except Exception:
    place['n_reviews'] = 0

try:
    place['n_photos'] = int(soup.find('div', class_='YkuOqf').text.replace('.', '').replace(',','').split(' ')[0])
except Exception:
    place['n_photos'] = 0

try:
    place['category'] = soup.find('button', jsaction='pane.rating.category').text.strip()
except Exception:
    place['category'] = None

try:
    place['description'] = soup.find('div', class_='PYvSYb').text.strip()
except Exception:
    place['description'] = None

b_list = soup.find_all('div', class_='Io6YTe fontBodyMedium kR99db')
try:
    place['address'] = b_list[0].text
except Exception:
    place['address'] = None

# Store the remaining details generically with error handling, as telephone, website vary in location and can not be reliably found using [n]
for i in range(1, len(b_list)):
    try:
        place[f'detail_{i+1}'] = b_list[i].text
    except Exception:
        place[f'detail_{i+1}'] = None

try:
    place['opening_hours'] = soup.find('div', class_='t39EBf GUrTXd')['aria-label'].replace('\u202f', ' ')
except:
    place['opening_hours'] = None

place['url'] = driver.current_url

lat, long, z = driver.current_url.split('/')[6].split(',')
place['lat'] = lat[1:]
place['long'] = long



In [82]:
place

{'name': 'Luciano’s Pizza',
 'overall_rating': 4.6,
 'n_reviews': 204,
 'n_photos': 32,
 'category': 'Pizza restaurant',
 'description': None,
 'address': '1005 Morris Park Ave # A, Bronx, NY 10462',
 'detail_2': '(718) 863-2850',
 'detail_3': 'R4XV+9G New York',
 'detail_4': 'Send to your phone',
 'detail_5': 'Claim this business',
 'opening_hours': 'Monday, 10 AM to 10 PM; Tuesday, 10 AM to 10 PM; Wednesday, 10 AM to 10 PM; Thursday, 10 AM to 10 PM; Friday, 10 AM to 10 PM; Saturday, 10 AM to 10 PM; Sunday, 10 AM to 10 PM. Hide open hours for the week',
 'url': 'https://www.google.com/maps/place/Luciano%E2%80%99s+Pizza/@40.848393,-73.856195,17z/data=!3m1!4b1!4m6!3m5!1s0x89c2f4a5aeb02823:0xc8368c15db491dd7!8m2!3d40.848393!4d-73.856195!16s%2Fg%2F1tcv_btn?entry=ttu',
 'lat': '40.848393',
 'long': '-73.856195'}

In [91]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time

# Function to scroll the reviews
def scroll_reviews(driver):
    try:
        # Locating the child element with classes 'e07Vkf' and 'kA9KIf' within the parent element with class 'w6VYqd'
        scrollable_div = driver.find_element(By.CSS_SELECTOR, '.w6VYqd div.e07Vkf.kA9KIf')
        # Attempting to scroll the identified element
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
            
    except Exception as e:
        print("Error scrolling:", e)

# Function to expand all reviews
def expand_reviews(driver):
    try:
        # Expand all reviews using the updated selector
        links = driver.find_elements(By.CSS_SELECTOR, 'button.M77dve[aria-label^="More reviews"]')
        for l in links:
            l.click()
        time.sleep(2)  # Adjust sleep time if necessary
    except Exception as e:
        print("Error expanding reviews:", e)

def expand_review_text(driver):
    # Find all review blocks
    review_blocks = driver.find_elements(By.CSS_SELECTOR, 'div.jftiEf.fontBodyMedium')

    # Iterate through each review block
    for review_block in review_blocks:
        try:
            # Within each review block, find the "More" button
            more_button = review_block.find_element(By.CSS_SELECTOR, 'button.w8nwRe.kyuRq')
            # Click the "More" button if found
            more_button.click()
        except NoSuchElementException:
            # If the "More" button is not found, continue to the next review block
            continue

In [84]:
scroll_reviews(driver)

In [86]:
expand_reviews(driver)

In [100]:
expand_review_text(driver)

In [109]:
from bs4 import BeautifulSoup

# Function to extract review information
def extract_reviews(soup, driver):
    reviews = []
    review_blocks = soup.find_all('div', class_='jftiEf fontBodyMedium')

    for review_block in review_blocks:
        # Call the function to expand the review text
        expand_review_text(driver)
        
        # Re-fetch and re-parse the page source to get updated content
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        review_block = soup.find('div', {'data-review-id': review_block['data-review-id']})
        
        review = {}

        try:
            review['username'] = review_block['aria-label']
        except Exception:
            review['username'] = None

        try:
            review['review_text'] = review_block.find('span', class_='wiI7pd').text
        except Exception:
            review['review_text'] = None

        try:
            review['rating'] = float(review_block.find('span', class_='kvMYJc')['aria-label'].split(' ')[0])
        except Exception:
            review['rating'] = None

        try:
            review['relative_date'] = review_block.find('span', class_='rsqaWe').text
        except Exception:
            review['relative_date'] = None

        reviews.append(review)
    
    return reviews

In [110]:
# Extract reviews
soup = BeautifulSoup(driver.page_source, 'html.parser')
reviews = extract_reviews(soup, driver)

In [111]:
print(reviews[0])

{'username': 'Robert Irizarry', 'review_text': 'Driving around 4 months ago with my wife and found this great pizza spot. It started with a regular slice each. Delicious!Then came back for a pie. Delicious!  Then decided to buy Margherita pie. Insanely super delicious! Have bought Margherita pie atleast 5x staight. I highly recommend this pizza spot. I will buy the grandma pizza next. Delicious 😋 You wont regret it.', 'rating': 5.0, 'relative_date': '5 months ago'}


In [ ]:
def expand_review_text(driver, review_block):
    try:
        # Find the 'More' button in the review block
        more_button = review_block.find_element_by_css_selector('button.w8nwRe.kyuRq')
        # Click the button if it's not already expanded
        if more_button.get_attribute('aria-expanded') == 'false':
            more_button.click()
        # Adjust sleep time if necessary
        time.sleep(1)
    except Exception as e:
        print("Error expanding review text:", e)

In [8]:
import openai
import env
openai.api_key = env.open_ai_key


In [25]:
prompt = '''
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.chrome import ChromeDriver, ChromeDriverManager
from bs4 import BeautifulSoup
from datetime import datetime
import time
import re
import logging
import traceback
import numpy as np
import itertools


GM_WEBPAGE = 'https://www.google.com/maps/'
MAX_WAIT = 10
MAX_RETRY = 5
MAX_SCROLLS = 40

class GoogleMapsScraper:

    def __init__(self, debug=False):
        self.debug = debug
        self.driver = self.__get_driver()
        self.logger = self.__get_logger()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, tb):
        if exc_type is not None:
            traceback.print_exception(exc_type, exc_value, tb)

        self.driver.close()
        self.driver.quit()

        return True

    def sort_by(self, url, ind):

        self.driver.get(url)
        self.__click_on_cookie_agreement()

        wait = WebDriverWait(self.driver, MAX_WAIT)

        # open dropdown menu
        clicked = False
        tries = 0
        while not clicked and tries < MAX_RETRY:
            try:
                menu_bt = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@data-value=\'Sort\']')))
                menu_bt.click()

                clicked = True
                time.sleep(3)
            except Exception as e:
                tries += 1
                self.logger.warn('Failed to click sorting button')

            # failed to open the dropdown
            if tries == MAX_RETRY:
                return -1

        #  element of the list specified according to ind
        recent_rating_bt = self.driver.find_elements_by_xpath('//div[@role=\'menuitemradio\']')[ind]
        recent_rating_bt.click()

        # wait to load review (ajax call)
        time.sleep(5)

        return 0

    def get_places(self, keyword_list=None):

        df_places = pd.DataFrame()
        search_point_url_list = self._gen_search_points_from_square(keyword_list=keyword_list)

        for i, search_point_url in enumerate(search_point_url_list):
            print(search_point_url)

            if (i+1) % 10 == 0:
                print(f"{i}/{len(search_point_url_list)}")
                df_places = df_places[['search_point_url', 'href', 'name', 'rating', 'num_reviews', 'close_time', 'other']]
                df_places.to_csv('output/places_wax.csv', index=False)


            try:
                self.driver.get(search_point_url)
            except NoSuchElementException:
                self.driver.quit()
                self.driver = self.__get_driver()
                self.driver.get(search_point_url)

            # scroll to load all (20) places into the page
            scrollable_div = self.driver.find_element_by_css_selector(
                "div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div[aria-label*='Results for']")
            for i in range(10):
                self.driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)

            # Get places names and href
            time.sleep(2)
            response = BeautifulSoup(self.driver.page_source, 'html.parser')
            div_places = response.select('div[jsaction] > a[href]')

            for div_place in div_places:
                place_info = {
                    'search_point_url': search_point_url.replace('https://www.google.com/maps/search/', ''),
                    'href': div_place['href'],
                    'name': div_place['aria-label']
                }

                df_places = df_places.append(place_info, ignore_index=True)

            # TODO: implement click to handle > 20 places

        df_places = df_places[['search_point_url', 'href', 'name']]
        df_places.to_csv('output/places_wax.csv', index=False)



    def get_reviews(self, offset):

        # scroll to load reviews
        self.__scroll()

        # wait for other reviews to load (ajax)
        time.sleep(4)

        # expand review text
        self.__expand_reviews()

        # parse reviews
        response = BeautifulSoup(self.driver.page_source, 'html.parser')
        # TODO: Subject to changes
        rblock = response.find_all('div', class_='jftiEf fontBodyMedium ')
        parsed_reviews = []
        for index, review in enumerate(rblock):
            if index >= offset:
                r = self.__parse(review)
                parsed_reviews.append(r)

                # logging to std out
                print(r)

        return parsed_reviews



    # need to use different url wrt reviews one to have all info
    def get_account(self, url):

        self.driver.get(url)
        self.__click_on_cookie_agreement()

        # ajax call also for this section
        time.sleep(2)

        resp = BeautifulSoup(self.driver.page_source, 'html.parser')

        place_data = self.__parse_place(resp, url)

        return place_data


    def __parse(self, review):

        item = {}

        try:
            # TODO: Subject to changes
            id_review = review['data-review-id']
        except Exception as e:
            id_review = None

        try:
            # TODO: Subject to changes
            username = review['aria-label']
        except Exception as e:
            username = None

        try:
            # TODO: Subject to changes
            review_text = self.__filter_string(review.find('span', class_='wiI7pd').text)
        except Exception as e:
            review_text = None

        try:
            # TODO: Subject to changes
            rating = float(review.find('span', class_='kvMYJc')['aria-label'].split(' ')[0])
        except Exception as e:
            rating = None

        try:
            # TODO: Subject to changes
            relative_date = review.find('span', class_='rsqaWe').text
        except Exception as e:
            relative_date = None

        try:
            n_reviews = review.find('div', class_='RfnDt').text.split(' ')[3]
        except Exception as e:
            n_reviews = 0

        try:
            user_url = review.find('button', class_='WEBjve')['data-href']
        except Exception as e:
            user_url = None

        item['id_review'] = id_review
        item['caption'] = review_text

        # depends on language, which depends on geolocation defined by Google Maps
        # custom mapping to transform into date should be implemented
        item['relative_date'] = relative_date

        # store datetime of scraping and apply further processing to calculate
        # correct date as retrieval_date - time(relative_date)
        item['retrieval_date'] = datetime.now()
        item['rating'] = rating
        item['username'] = username
        item['n_review_user'] = n_reviews
        #item['n_photo_user'] = n_photos  ## not available anymore
        item['url_user'] = user_url

        return item


    def __parse_place(self, response, url):

        place = {}

        try:
            place['name'] = response.find('h1', class_='DUwDvf fontHeadlineLarge').text.strip()
        except Exception as e:
            place['name'] = None

        try:
            place['overall_rating'] = float(response.find('div', class_='F7nice ').find('span', class_='ceNzKf')['aria-label'].split(' ')[1])
        except Exception as e:
            place['overall_rating'] = None

        try:
            place['n_reviews'] = int(response.find('div', class_='F7nice ').text.split('(')[1].replace(',', '').replace(')', ''))
        except Exception as e:
            place['n_reviews'] = 0

        try:
            place['n_photos'] = int(response.find('div', class_='YkuOqf').text.replace('.', '').replace(',','').split(' ')[0])
        except Exception as e:
            place['n_photos'] = 0

        try:
            place['category'] = response.find('button', jsaction='pane.rating.category').text.strip()
        except Exception as e:
            place['category'] = None

        try:
            place['description'] = response.find('div', class_='PYvSYb').text.strip()
        except Exception as e:
            place['description'] = None

        b_list = response.find_all('div', class_='Io6YTe fontBodyMedium')
        try:
            place['address'] = b_list[0].text
        except Exception as e:
            place['address'] = None

        try:
            place['website'] = b_list[1].text
        except Exception as e:
            place['website'] = None

        try:
            place['phone_number'] = b_list[2].text
        except Exception as e:
            place['phone_number'] = None
    
        try:
            place['plus_code'] = b_list[3].text
        except Exception as e:
            place['plus_code'] = None

        try:
            place['opening_hours'] = response.find('div', class_='t39EBf GUrTXd')['aria-label'].replace('\u202f', ' ')
        except:
            place['opening_hours'] = None

        place['url'] = url

        lat, long, z = url.split('/')[6].split(',')
        place['lat'] = lat[1:]
        place['long'] = long

        return place


    def _gen_search_points_from_square(self, keyword_list=None):
        # TODO: Generate search points from corners of square

        keyword_list = [] if keyword_list is None else keyword_list

        square_points = pd.read_csv('input/square_points.csv')

        cities = square_points['city'].unique()

        search_urls = []

        for city in cities:

            df_aux = square_points[square_points['city'] == city]
            latitudes = df_aux['latitude'].unique()
            longitudes = df_aux['longitude'].unique()
            coordinates_list = list(itertools.product(latitudes, longitudes, keyword_list))

            search_urls += [f"https://www.google.com/maps/search/{coordinates[2]}/@{str(coordinates[1])},{str(coordinates[0])},{str(15)}z"
             for coordinates in coordinates_list]

        return search_urls


    # expand review description
    def __expand_reviews(self):
        # use XPath to load complete reviews
        # TODO: Subject to changes
        links = self.driver.find_elements_by_xpath('//button[@jsaction="pane.review.expandReview"]')
        for l in links:
            l.click()
        time.sleep(2)


    def __scroll(self):
        # TODO: Subject to changes
        scrollable_div = self.driver.find_element_by_css_selector('div.m6QErb.DxyBCb.kA9KIf.dS8AEf')
        self.driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        #self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


    def __get_logger(self):
        # create logger
        logger = logging.getLogger('googlemaps-scraper')
        logger.setLevel(logging.DEBUG)

        # create console handler and set level to debug
        fh = logging.FileHandler('gm-scraper.log')
        fh.setLevel(logging.DEBUG)

        # create formatter
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

        # add formatter to ch
        fh.setFormatter(formatter)

        # add ch to logger
        logger.addHandler(fh)

        return logger


    def __get_driver(self, debug=False):
        options = Options()

        if not self.debug:
            options.add_argument("--headless")
        else:
            options.add_argument("--window-size=1366,768")

        options.add_argument("--disable-notifications")
        #options.add_argument("--lang=en-GB")
        options.add_argument("--accept-lang=en-GB")
        
        # Specify the path to the Chrome Beta binary
        chrome_binary_path = "/Applications/Google Chrome Beta.app/Contents/MacOS/Google Chrome Beta"

        # Set the path to the Chrome Beta binary
        options.binary_location = chrome_binary_path
        
        # Specify the path to the downloaded ChromeDriver executable for Chrome Beta
        driver_path = "/Users/jongarcia/codeup-data-science/googlemaps-scraper/chromedriver-mac-x64 BETA/chromedriver"

        input_driver = webdriver.Chrome(executable_path=driver_path, options=options)

        input_driver.get(GM_WEBPAGE)

        return input_driver





    # cookies agreement click
    def __click_on_cookie_agreement(self):
        try:
            agree = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//span[contains(text(), "Reject all")]')))
            agree.click()

            # back to the main page
            # self.driver.switch_to_default_content()

            return True
        except:
            return False

    # util function to clean special characters
    def __filter_string(self, str):
        strOut = str.replace('\r', ' ').replace('\n', ' ').replace('\t', ' ')
        return strOut

Above is the googlemaps.py file
Below is the scraper.py file

# -*- coding: utf-8 -*-
from googlemaps import GoogleMapsScraper
from datetime import datetime, timedelta
import argparse
import csv
from termcolor import colored
import time


ind = {'most_relevant' : 0 , 'newest' : 1, 'highest_rating' : 2, 'lowest_rating' : 3 }
HEADER = ['id_review', 'caption', 'relative_date', 'retrieval_date', 'rating', 'username', 'n_review_user', 'n_photo_user', 'url_user']
HEADER_W_SOURCE = ['id_review', 'caption', 'relative_date','retrieval_date', 'rating', 'username', 'n_review_user', 'n_photo_user', 'url_user', 'url_source']

def csv_writer(source_field, ind_sort_by, path='data/'):
    outfile= ind_sort_by + '_gm_reviews.csv'
    targetfile = open(path + outfile, mode='w', encoding='utf-8', newline='\n')
    writer = csv.writer(targetfile, quoting=csv.QUOTE_MINIMAL)

    if source_field:
        h = HEADER_W_SOURCE
    else:
        h = HEADER
    writer.writerow(h)

    return writer


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Google Maps reviews scraper.')
    parser.add_argument('--N', type=int, default=100, help='Number of reviews to scrape')
    parser.add_argument('--i', type=str, default='urls.txt', help='target URLs file')
    parser.add_argument('--sort_by', type=str, default='newest', help='most_relevant, newest, highest_rating or lowest_rating')
    parser.add_argument('--place', dest='place', action='store_true', help='Scrape place metadata')
    parser.add_argument('--debug', dest='debug', action='store_true', help='Run scraper using browser graphical interface')
    parser.add_argument('--source', dest='source', action='store_true', help='Add source url to CSV file (for multiple urls in a single file)')
    parser.set_defaults(place=False, debug=False, source=False)

    args = parser.parse_args()

    # store reviews in CSV file
    writer = csv_writer(args.source, args.sort_by)

    with GoogleMapsScraper(debug=args.debug) as scraper:
        with open(args.i, 'r') as urls_file:
            for url in urls_file:
                if args.place:
                    print(scraper.get_account(url))
                else:
                    error = scraper.sort_by(url, ind[args.sort_by])

                    if error == 0:

                        n = 0

                        #if ind[args.sort_by] == 0:
                        #    scraper.more_reviews()

                        while n < args.N:

                            # logging to std out
                            print(colored('[Review ' + str(n) + ']', 'cyan'))

                            reviews = scraper.get_reviews(n)
                            if len(reviews) == 0:
                                break

                            for r in reviews:
                                row_data = list(r.values())
                                if args.source:
                                    row_data.append(url[:-1])

                                writer.writerow(row_data)

                            n += len(reviews)

The below code was tested today in order to help you upgrade the googlemaps.py file:

from selenium import webdriver 
# Use the driver below instead of using ChromeDriverManager to select the driver
# Specify the path to the Chrome Beta binary
chrome_binary_path = "/Applications/Google Chrome Beta.app/Contents/MacOS/Google Chrome Beta"

# Specify the path to the ChromeDriver executable for Chrome Beta
driver_path = "/Users/jongarcia/codeup-data-science/googlemaps-scraper/chromedriver-mac-x64 BETA/chromedriver"

# Create a ChromeOptions object
options = webdriver.ChromeOptions()

# Set the binary location to Chrome Beta
options.binary_location = chrome_binary_path

# Create a ChromeDriver with the specified options and executable path
driver = webdriver.Chrome(executable_path=driver_path, options=options)

from bs4 import BeautifulSoup
import time

soup = BeautifulSoup(driver.page_source, 'html.parser')

place = {}

try:
    place['name'] = soup.find('h1', class_='DUwDvf lfPIob').text.strip()
except Exception:
    place['name'] = None

try:
    place['overall_rating'] = float(soup.find('div', class_='F7nice').find('span', class_='ceNzKf')['aria-label'].split(' ')[0])
except Exception:
    place['overall_rating'] = None

try:
    place['n_reviews'] = int(soup.find('div', class_='F7nice').text.split('(')[1].replace(',', '').replace(')', ''))
except Exception:
    place['n_reviews'] = 0

try:
    place['n_photos'] = int(soup.find('div', class_='YkuOqf').text.replace('.', '').replace(',','').split(' ')[0])
except Exception:
    place['n_photos'] = 0

try:
    place['category'] = soup.find('button', jsaction='pane.rating.category').text.strip()
except Exception:
    place['category'] = None

try:
    place['description'] = soup.find('div', class_='PYvSYb').text.strip()
except Exception:
    place['description'] = None

b_list = soup.find_all('div', class_='Io6YTe fontBodyMedium kR99db')
try:
    place['address'] = b_list[0].text
except Exception:
    place['address'] = None

# Store the remaining details generically with error handling, as telephone, website vary in location and can not be reliably found using [n]
for i in range(1, len(b_list)):
    try:
        place[f'detail_{i+1}'] = b_list[i].text
    except Exception:
        place[f'detail_{i+1}'] = None

try:
    place['opening_hours'] = soup.find('div', class_='t39EBf GUrTXd')['aria-label'].replace('\u202f', ' ')
except:
    place['opening_hours'] = None

place['url'] = driver.current_url

lat, long, z = driver.current_url.split('/')[6].split(',')
place['lat'] = lat[1:]
place['long'] = long

IMPORTANT the functions below are meant to show you the updated html but you should weight strongly the original logic in the py files

# Function to scroll the reviews
def scroll_reviews(driver):
    try:
        # Locating the scrollable div for reviews
        scrollable_div = driver.find_element_by_css_selector('div.m6QErb.DxyBCb.kA9KIf.dS8AEf')
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
    except Exception as e:
        print("Error scrolling:", e)

# Function to expand all reviews
def expand_reviews(driver):
    try:
        # Expand all reviews using the updated selector
        links = driver.find_elements_by_css_selector('button.M77dve[aria-label^="More reviews"]')
        # links = driver.find_elements_by_xpath('//button[@jsaction="pane.review.expandReview"]')
        for l in links:
            l.click()
        time.sleep(2)  # Adjust sleep time if necessary
    except Exception as e:
        print("Error expanding reviews:", e)

def expand_review_text(driver, review_block):
    try:
        # Find the first "More" button within the review block
        more_button = review_block.find('button', class_='w8nwRe kyuRq')

        if more_button:
            # Extract the unique data-review-id attribute from the button
            data_review_id = more_button['data-review-id']

            # Use Selenium to find the button using the data-review-id attribute
            more_button_selenium = driver.find_element_by_css_selector(f'button[data-review-id="{data_review_id}"].w8nwRe.kyuRq')

            if more_button_selenium:
                more_button_selenium.click()  # Click the button to expand the review text
                time.sleep(1)  # Adjust this delay as necessary to allow the text to expand
    except Exception as e:
        print(f"Error expanding review text: {e}")

# Function to extract review information
def extract_reviews(soup):
    reviews = []
    review_blocks = soup.find_all('div', class_='jftiEf fontBodyMedium')
    for review_block in review_blocks:
        # Call the function to expand the review text
        expand_review_text(driver, review_block)
        
        # Re-fetch and re-parse the page source to get updated content
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        review_block = soup.find('div', {'data-review-id': review_block['data-review-id']})
        
        review = {}

        try:
            review['username'] = review_block['aria-label']
        except Exception:
            review['username'] = None

        try:
            review['review_text'] = review_block.find('span', class_='wiI7pd').text
        except Exception:
            review['review_text'] = None

        try:
            review['rating'] = float(review_block.find('span', class_='kvMYJc')['aria-label'].split(' ')[0])
        except Exception:
            review['rating'] = None

        try:
            review['relative_date'] = review_block.find('span', class_='rsqaWe').text
        except Exception:
            review['relative_date'] = None

        reviews.append(review)
    
    return reviews


as you can see the scraper.py file  calls on the googlemaps.py file to scrape based on a url txt file. 

the googlemaps.py file hold outdated scraping html elements but the logic is sound. 
some logic is no longer useful as the libraries have deprecated some functions
currently there is an issue with ChromeDriverManager, it can not be used

Steps to Update the Code:

1.Update HTML Selectors:

Review the updated code snippets you've provided and identify the new HTML selectors.
Update the selectors in the googlemaps.py file wherever the HTML structure has changed.

2. Remove ChromeDriverManager:

In the __get_driver method of the GoogleMapsScraper class, replace the use of ChromeDriverManager with the approach you've tested.
Make sure to handle the paths to the Chrome Beta binary and ChromeDriver executable dynamically or make them configurable.

3. Refactor Functions:

Do not replace the original functions 1:1  like __scroll with scroll_reviews. Instead remember that the logic in the original py files is better, and the new functions are mainly to test the current selectors. 

additional notes:
1) I do not need instructions or explanations on how to update the code, you will be performing the task of updating the code. 
2) I am not asking you to execute, or run code.
3) you will re-write all of the code in the googlemaps.py do not use comments instead of re-writing the code.
4)  
'''



In [26]:
def generate_linkedin_connection_request():
    completion = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {
                "role": "user",
                "content": f"{prompt}"
            }
        ]
    )

    return completion.choices[0].message['content']

In [27]:
reply = generate_linkedin_connection_request()
print(reply)

Sure, from your explanation I see you'd like to update the original code to accommodate new HTML selectors, remove ChromeDriverManager usage, and refactor methods. This update will ensure the code is using the updated version of the library and accurate HTML selectors. 

Below is the updated `googlemaps.py` file:

```python
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import csv

class GoogleMapsScraper:

    def __init__(self, debug=False):

        path_for_chromeDriver = "/path/to/chromedriver"
        # Use a specific version of chrome not default version
        chrome_binary_path = "/path/to/Google Chrome Beta.app/Contents/Mac

In [14]:
import requests

# Replace 'YOUR_API_KEY' with your actual OpenAI API key
api_key = env.open_ai_key

# Set the API endpoint URL
url = 'https://api.openai.com/v1/models'

# Set the headers with your API key
headers = {
    'Authorization': f'Bearer {api_key}'
}

# Make the GET request to the API
response = requests.get(url, headers=headers)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    models = response.json()
    for model in models['data']:
        print(f"Model Name: {model['id']}")
else:
    print(f"Error: {response.status_code} - {response.text}")


Model Name: text-search-babbage-doc-001
Model Name: gpt-3.5-turbo-16k-0613
Model Name: curie-search-query
Model Name: gpt-3.5-turbo-16k
Model Name: text-search-babbage-query-001
Model Name: babbage
Model Name: babbage-search-query
Model Name: text-babbage-001
Model Name: whisper-1
Model Name: text-similarity-davinci-001
Model Name: davinci-similarity
Model Name: code-davinci-edit-001
Model Name: curie-similarity
Model Name: babbage-search-document
Model Name: curie-instruct-beta
Model Name: text-search-ada-doc-001
Model Name: davinci-instruct-beta
Model Name: gpt-3.5-turbo-0613
Model Name: text-similarity-babbage-001
Model Name: text-search-davinci-doc-001
Model Name: gpt-4-0314
Model Name: gpt-4-0613
Model Name: gpt-4
Model Name: babbage-similarity
Model Name: text-embedding-ada-002
Model Name: davinci-search-query
Model Name: text-similarity-curie-001
Model Name: text-davinci-001
Model Name: text-search-davinci-query-001
Model Name: ada-search-document
Model Name: ada-code-search-cod

In [13]:
models

{'object': 'list',
 'data': [{'id': 'text-search-babbage-doc-001',
   'object': 'model',
   'created': 1651172509,
   'owned_by': 'openai-dev',
   'permission': [{'id': 'modelperm-s9n5HnzbtVn7kNc5TIZWiCFS',
     'object': 'model_permission',
     'created': 1695933794,
     'allow_create_engine': False,
     'allow_sampling': True,
     'allow_logprobs': True,
     'allow_search_indices': True,
     'allow_view': True,
     'allow_fine_tuning': False,
     'organization': '*',
     'group': None,
     'is_blocking': False}],
   'root': 'text-search-babbage-doc-001',
   'parent': None},
  {'id': 'gpt-3.5-turbo-16k-0613',
   'object': 'model',
   'created': 1685474247,
   'owned_by': 'openai',
   'permission': [{'id': 'modelperm-6TvrqdxTZPTKF0DrBbCYP3sh',
     'object': 'model_permission',
     'created': 1697854292,
     'allow_create_engine': False,
     'allow_sampling': True,
     'allow_logprobs': True,
     'allow_search_indices': False,
     'allow_view': True,
     'allow_fine_tu